[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/pixeltable/pixeltable/blob/release/docs/notebooks/use-cases/rag-operations.ipynb)&nbsp;&nbsp;
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixeltable/pixeltable/blob/release/docs/notebooks/use-cases/rag-operations.ipynb)

# RAG Operations in Pixeltable

In this tutorial, we'll explore Pixeltable's flexible handling of RAG operations on unstructured text. In a traditional AI workflow, such operations might be implemented as a Python script that runs on a periodic schedule or in response to certain events. In Pixeltable, as with everything else, they are implemented as persistent table operations that update incrementally as new data becomes available. In our tutorial workflow, we'll chunk Wikipedia articles in various ways with a document splitter, then apply several kinds of embeddings to the chunks.

## Set Up the Table Structure

We start by installing the necessary dependencies, creating a Pixeltable directory `rag_ops_demo` (if it doesn't already exist), and setting up the table structure for our new workflow.

In [ ]:
%pip install -q pixeltable sentence-transformers spacy tiktoken

In [1]:
import pixeltable as pxt

# Ensure a clean slate for the demo
pxt.drop_dir('rag_ops_demo', force=True)
# Create the Pixeltable workspace
pxt.create_dir('rag_ops_demo')

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/asiegel/.pixeltable/pgdata
Created directory `rag_ops_demo`.


## Creating Tables and Views

Now we'll create the tables that represent our workflow, starting with a table to hold references to source documents. The table contains a single column `source_doc` whose elements have type `pxt.Document`, representing a general document instance. In this tutorial, we'll be working with HTML documents, but Pixeltable supports a range of other document types, such as Markdown and PDF.

In [2]:
docs = pxt.create_table(
    'rag_ops_demo.docs',
    {'source_doc': pxt.Document}
)

Created table `docs`.


If we take a peek at the `docs` table, we see its very simple structure.

In [3]:
docs

Table
'rag_ops_demo.docs'

 Column Name      Type Computed With
  source_doc  Document

Next we create a view to represent chunks of our HTML documents. A Pixeltable view is a virtual table, which is dynamically derived from a source table by applying a transformation and/or selecting a subset of data. In this case, our view represents a one-to-many transformation from source documents into individual sentences. This is achieved using Pixeltable's built-in `DocumentSplitter` class.

Note that the `docs` table is currently empty, so creating this view doesn't actually *do* anything yet: it simply defines an operation that we want Pixeltable to execute when it sees new data.

In [4]:
from pixeltable.iterators.document import DocumentSplitter

sentences = pxt.create_view(
    'rag_ops_demo.sentences',  # Name of the view
    docs,  # Table from which the view is derived
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='sentence',  # Chunk docs into sentences
        metadata='title,heading,sourceline'
    )
)

Created view `sentences` with 0 rows, 0 exceptions.


Let's take a peek at the new `sentences` view.

In [5]:
sentences

View
'rag_ops_demo.sentences'
(of 'rag_ops_demo.docs')

 Column Name              Type Computed With
         pos     Required[Int]              
        text  Required[String]              
       title            String              
     heading              Json              
  sourceline               Int              
  source_doc          Document

We see that `sentences` inherits the `source_doc` column from `docs`, together with some new fields:
- `pos`: The position in the source document where the sentence appears.
-  `text`: The text of the sentence.
- `title`, `heading`, and `sourceline`: The metadata we requested when we set up the view.

## Data Ingestion

Ok, now it's time to insert some data into our workflow. A document in Pixeltable is just a URL; the following command inserts a single row into the `docs` table with the `source_doc` field set to the specified URL:

In [6]:
docs.insert([{'source_doc': 'https://en.wikipedia.org/wiki/Marc_Chagall'}])

Computing cells: 100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 14.50 cells/s]
Inserting rows into `docs`: 1 rows [00:00, 739.08 rows/s]
Computing cells: 100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 14.20 cells/s]
Inserting rows into `sentences`: 1460 rows [00:00, 3529.45 rows/s]
Inserted 1461 rows with 0 errors.


UpdateStatus(num_rows=1461, num_computed_values=2, num_excs=0, updated_cols=[], cols_with_excs=[])

We can see that two things happened. First, a single row was inserted into `docs`, containing the URL representing our source document. Then, the view `sentences` was incrementally updated by applying the `DocumentSplitter` according to the definition of the view. This illustrates an important principle in Pixeltable: by default, anytime Pixeltable sees new data, the update is incrementally propagated to any downstream views or computed columns.

We can see the effect of the insertion with the `select` command. There's a single row in `docs`:

In [7]:
docs.select(docs.source_doc.fileurl).show()

source_doc_fileurl
https://en.wikipedia.org/wiki/Marc_Chagall


And here are the first 20 rows in `sentences`. The content of the article is broken into individual sentences, as expected.

In [8]:
sentences.select(sentences.text, sentences.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية,"{""h1"": ""Marc Chagall""}"
Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी,"{""h1"": ""Marc Chagall""}"
Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย,"{""h1"": ""Marc Chagall""}"
Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש,"{""h1"": ""Marc Chagall""}"
"粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here.","{""h1"": ""Marc Chagall""}"
"For other uses, see Chagall (disambiguation) .","{""h1"": ""Marc Chagall""}"
"Marc Chagall Chagall, c. 1920","{""h1"": ""Marc Chagall""}"
"Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus)","{""h1"": ""Marc Chagall""}"
[1] Died 28 March 1985 (1985-03-28) (aged 97),"{""h1"": ""Marc Chagall""}"


## Experimenting with Chunking

Of course, chunking into sentences isn't the only way to split a document. Perhaps we want to experiment with different chunking methodologies, in order to see which one performs best in a particular application. Pixeltable makes it easy to do this, by creating several views of the same source table. Here are a few examples. Notice that as each new view is created, it is initially populated from the data already in `docs`.

In [9]:
chunks = pxt.create_view(
    'rag_ops_demo.chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,token_limit',
        limit=2048,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `chunks`: 205 rows [00:00, 16715.90 rows/s]
Created view `chunks` with 205 rows, 0 exceptions.


In [10]:
short_chunks = pxt.create_view(
    'rag_ops_demo.short_chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,token_limit',
        limit=72,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `short_chunks`: 531 rows [00:00, 23858.59 rows/s]
Created view `short_chunks` with 531 rows, 0 exceptions.


In [11]:
short_char_chunks = pxt.create_view(
    'rag_ops_demo.short_char_chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,char_limit',
        limit=72,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `short_char_chunks`: 1764 rows [00:00, 17427.53 rows/s]
Created view `short_char_chunks` with 1764 rows, 0 exceptions.


In [12]:
chunks.select(chunks.text, chunks.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .","{""h1"": ""Marc Chagall""}"
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus) [1] Died 28 March 1985 (1985-03-28) (aged 97) Saint-Paul-de-Vence , France Nationality Russian Empire, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Valentina (Vava) Brodsky ​ ​ ( m. 1952) ​ [3] Children 2 [4]","{""h1"": ""Marc Chagall""}"
"Marc Chagall [a] (born Moishe Shagal ; 6 July [ O.S. 24 June] 1887 – 28 March 1985) was a Belarusian-French artist. [b] An early modernist , he was associated with the École de Paris as well as several major artistic styles and created works in a wide range of artistic formats, including painting, drawings, book illustrations, stained glass , stage sets, ceramics, tapestries and fine art prints.","{""h1"": ""Marc Chagall""}"
"Chagall was born in 1887, into a Jewish family near Vitebsk , today in Belarus , but at that time in the Pale of Settlement of the Russian Empire. Before World War I , he travelled between Saint Petersburg , Paris , and Berlin . During that period, he created his own mixture and style of modern art, based on his ideas of Eastern European and Jewish folklore. He spent the wartime years in his native Belarus, becoming one of the country's most distinguished artists and a member of the modernist avant-garde , founding the Vitebsk Arts College . He later worked in and near Moscow in difficult conditions during hard times in Russia following the Bolshevik Revolution , before leaving again for Paris in 1923. During World War II , he escaped occupied France to the United States, where he lived in New York City for seven years before returning to France in 1948.","{""h1"": ""Marc Chagall""}"
"Art critic Robert Hughes referred to Chagall as ""the quintessential Jewish artist of the twentieth century"". According to art historian Michael J. Lewis, Chagall was considered to be ""the last survivor of the first generation of European modernists"". For decades, he ""had also been respected as the world's pre-eminent Jewish artist"". [15] Using the medium of stained glass, he produced windows for the cathedrals of Reims and Metz as well as the Fraumünster in Zürich , windows for the UN and th ...... e experienced modernism's ""golden age"" in Paris, where ""he synthesized the art forms of Cubism , Symbolism , and Fauvism , and the influence of Fauvism gave rise to Surrealism "". Yet throughout these phases of his style ""he remained most emphatically a Jewish artist, whose work was one long dreamy reverie of life in his native village of Vitebsk."" [16] ""When Matisse dies"", Pablo Picasso remarked in the 1950s, ""Chagall will be the only painter left who understands what colour really is"". [17]","{""h1"": ""Marc Chagall""}"
Early life and education [ edit ],"{""h1"": 

In [13]:
short_chunks.select(short_chunks.text, short_chunks.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡ,"{""h1"": ""Marc Chagall""}"
ортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한,"{""h1"": ""Marc Chagall""}"
국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lë,"{""h1"": ""Marc Chagall""}"
tzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemont,"{""h1"": ""Marc Chagall""}"
èis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi,"{""h1"": ""Marc Chagall""}"
"Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here","{""h1"": ""Marc Chagall""}"
". For other uses, see Chagall (disambiguation) .","{""h1"": ""Marc Chagall""}"
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus) [1] Died 28 March 1985 (1985-03-","{""h1"": ""Marc Chagall""}"
"28) (aged 97) Saint-Paul-de-Vence , France Nationality Russian Empire, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Val","{""h1"": ""Marc Chagall""}"


In [14]:
short_char_chunks.select(short_char_chunks.text, short_char_chunks.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտա,"{""h1"": ""Marc Chagall""}"
հայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (та,"{""h1"": ""Marc Chagall""}"
рашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά,"{""h1"": ""Marc Chagall""}"
Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrva,"{""h1"": ""Marc Chagall""}"
tski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswah,"{""h1"": ""Marc Chagall""}"
ili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy م,"{""h1"": ""Marc Chagall""}"
صرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbe,"{""h1"": ""Marc Chagall""}"
kcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Ro,"{""h1"": ""Marc Chagall""}"
mână Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina S,"{""h1"": ""Marc Chagall""}"


Now let's add a few more documents to our workflow. Notice how all of the downstream views are updated incrementally, processing just the new documents as they are inserted.

In [15]:
urls = [
    'https://en.wikipedia.org/wiki/Pierre-Auguste_Renoir',
    'https://en.wikipedia.org/wiki/Henri_Matisse',
    'https://en.wikipedia.org/wiki/Marcel_Duchamp'
]
docs.insert({'source_doc': url} for url in urls)

Computing cells: 100%|████████████████████████████████████████████| 6/6 [00:00<00:00, 53.63 cells/s]
Inserting rows into `docs`: 3 rows [00:00, 2365.21 rows/s]
Computing cells: 100%|████████████████████████████████████████████| 6/6 [00:00<00:00, 52.39 cells/s]
Inserting rows into `sentences`: 2106 rows [00:02, 783.63 rows/s]
Inserting rows into `chunks`: 276 rows [00:00, 15888.61 rows/s]
Inserting rows into `short_chunks`: 812 rows [00:00, 22184.42 rows/s]
Inserting rows into `short_char_chunks`: 2638 rows [00:00, 13227.11 rows/s]
Inserted 5835 rows with 0 errors.


UpdateStatus(num_rows=5835, num_computed_values=6, num_excs=0, updated_cols=[], cols_with_excs=[])


Inserted 5831 rows with 0 errors.


UpdateStatus(num_rows=5831, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

## Further Experiments

This is a good time to mention another important guiding principle of Pixeltable. The preceding examples all used the built-in `DocumentSplitter` class with various configurations. That's probably fine as a first cut or to prototype an application quickly, and it might be sufficient for some applications. But other applications might want to do more sophisticated kinds of chunking, implementing their own specialized logic or leveraging third-party tools. Pixeltable imposes no constraints on the AI or RAG operations a workflow uses: the iterator interface is highly general, and it's easy to implement new operations or adapt existing code or third-party tools into the Pixeltable workflow.

## Computing Embeddings

Next, let's look at how embedding indices can be added seamlessly to existing Pixeltable workflows. To compute our embeddings, we'll use the Huggingface `sentence_transformer` package, running it over the `chunks` view that broke our documents up into larger paragraphs. Pixeltable has a built-in `sentence_transformer` adapter, and all we have to do is add a new column that leverages it. Pixeltable takes care of the rest, applying the new column to all existing data in the view.

In [16]:
from pixeltable.functions.huggingface import sentence_transformer

chunks.add_computed_column(minilm_embed=sentence_transformer(
    chunks.text,
    model_id='paraphrase-MiniLM-L6-v2'
))

Computing cells: 100%|███████████████████████████████████████| 481/481 [00:02<00:00, 222.59 cells/s]
Added 481 column values with 0 errors.


UpdateStatus(num_rows=481, num_computed_values=481, num_excs=0, updated_cols=[], cols_with_excs=[])

The new column is a *computed column*: it is defined as a function on top of existing data and updated incrementally as new data are added to the workflow. Let's have a look at how the new column affected the `chunks` view.

In [17]:
chunks

View
'rag_ops_demo.chunks'
(of 'rag_ops_demo.docs')

   Column Name                            Type                                      Computed With
           pos                   Required[Int]                                                   
          text                Required[String]                                                   
         title                          String                                                   
       heading                            Json                                                   
    sourceline                             Int                                                   
  minilm_embed  Required[Array[(384,), Float]]  sentence_transformer(text, model_id='paraphras...
    source_doc                        Document

In [18]:
chunks.select(chunks.text, chunks.heading, chunks.minilm_embed).head()

text,heading,minilm_embed
Marc Chagall - Wikipedia Jump to content Search Search,{},[-0.262 -0.119 -0.133 0.048 0.12 -0.006 ... -0.556 0.372 0.468 -0.234 -0.226 0.164]
"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .","{""h1"": ""Marc Chagall""}",[-0.136 0.401 -0.53 -0.181 -0.453 -0.125 ... -0.184 0.122 0.644 -0.54 0.188 0.203]
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus) [1] Died 28 March 1985 (1985-03-28) (aged 97) Saint-Paul-de-Vence , France Nationality Russian Empire, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Valentina (Vava) Brodsky ​ ​ ( m. 1952) ​ [3] Children 2 [4]","{""h1"": ""Marc Chagall""}",[-0.005 0.34 -0.315 0.17 -0.124 0.384 ... -0.144 -0.131 0.104 -0.412 -0.195 -0.058]
"Marc Chagall [a] (born Moishe Shagal ; 6 July [ O.S. 24 June] 1887 – 28 March 1985) was a Belarusian-French artist. [b] An early modernist , he was associated with the École de Paris as well as several major artistic styles and created works in a wide range of artistic formats, including painting, drawings, book illustrations, stained glass , stage sets, ceramics, tapestries and fine art prints.","{""h1"": ""Marc Chagall""}",[ 0.053 0.138 -0.219 0.192 -0.1 0.234 ... -0.138 -0.294 0.306 -0.012 -0.059 0.007]
"Chagall was born in 1887, into a Jewish family near Vitebsk , today in Belarus , but at that time in the Pale of Settlement of the Russian Empire. Before World War I , he travelled between Saint Petersburg , Paris , and Berlin . During that period, he created his own mixture and style of modern art, based on his ideas of Eastern European and Jewish folklore. He spent the wartime years in his native Belarus, becoming one of the country's most distinguished artists and a member of the modernist avant-garde , founding the Vitebsk Arts College . He later worked in and near Moscow in difficult conditions during hard times in Russia following the Bolshevik Revolution , before leaving again for Paris in 1923. During World War II , he escaped occupied France to the United States, where he lived in New York City for seven years before returning to France in 1948.","{""h1"": ""Marc Chagall""}",[ 0.013 0.248 -0.692 0.143 -0.379 0.254 ... -0.232 -0.157 -0.018 -0.225 -0.208 -0.095]
"Art critic Robert Hughes referred to Chagall as ""the quintessential Jewish artist of the twentieth century"". According to art historian Michael J. Lewis, Chagall was considered to be ""the last survivor of the first generation of European modernists"". For decades, he ""had also been respected as the world's pre-eminent Jewish artist"". [15] Using the medium of stained glass, he produced windows for the cathedrals of Reims and Metz as well as the Fraumünster in Zürich , windows for the UN and th ...... e experienced modernism's ""golden age"" in Paris, where ""he synthesized the art forms of Cubism , Symbolism , and Fauvism , and the influence of 

Similarly, we might want to add a CLIP embedding to our workflow; once again, it's just another computed column:

In [19]:
from pixeltable.functions.huggingface import clip

chunks.add_computed_column(clip_embed=clip(
    chunks.text, model_id='openai/clip-vit-base-patch32'
))

Computing cells: 100%|████████████████████████████████████████| 481/481 [00:05<00:00, 93.49 cells/s]
Added 481 column values with 0 errors.


UpdateStatus(num_rows=481, num_computed_values=481, num_excs=0, updated_cols=[], cols_with_excs=[])

In [20]:
chunks

View
'rag_ops_demo.chunks'
(of 'rag_ops_demo.docs')

   Column Name                            Type                                      Computed With
           pos                   Required[Int]                                                   
          text                Required[String]                                                   
         title                          String                                                   
       heading                            Json                                                   
    sourceline                             Int                                                   
  minilm_embed  Required[Array[(384,), Float]]  sentence_transformer(text, model_id='paraphras...
    clip_embed  Required[Array[(512,), Float]]  clip(text, model_id='openai/clip-vit-base-patc...
    source_doc                        Document

In [21]:
chunks.select(chunks.text, chunks.heading, chunks.clip_embed).head()

text,heading,clip_embed
Marc Chagall - Wikipedia Jump to content Search Search,{},[ 0.439 -0.204 -0.37 -0.248 0.253 0.11 ... 0.216 -0.31 0.22 0.557 0.173 -0.271]
"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .","{""h1"": ""Marc Chagall""}",[ 0.106 0.006 -0.152 0.043 0.283 -0.017 ... 0.237 -0.424 0.203 0.016 0.117 -0.32 ]
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus) [1] Died 28 March 1985 (1985-03-28) (aged 97) Saint-Paul-de-Vence , France Nationality Russian Empire, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Valentina (Vava) Brodsky ​ ​ ( m. 1952) ​ [3] Children 2 [4]","{""h1"": ""Marc Chagall""}",[ 0.301 -0.216 -0.186 -0.098 0.273 0.154 ... 0.051 -0.35 0.324 0.152 0.156 -0.235]
"Marc Chagall [a] (born Moishe Shagal ; 6 July [ O.S. 24 June] 1887 – 28 March 1985) was a Belarusian-French artist. [b] An early modernist , he was associated with the École de Paris as well as several major artistic styles and created works in a wide range of artistic formats, including painting, drawings, book illustrations, stained glass , stage sets, ceramics, tapestries and fine art prints.","{""h1"": ""Marc Chagall""}",[ 0.291 -0.174 -0.137 -0.067 0.438 0.076 ... 0.158 -0.32 0.431 -0.274 0.152 -0.207]
"Chagall was born in 1887, into a Jewish family near Vitebsk , today in Belarus , but at that time in the Pale of Settlement of the Russian Empire. Before World War I , he travelled between Saint Petersburg , Paris , and Berlin . During that period, he created his own mixture and style of modern art, based on his ideas of Eastern European and Jewish folklore. He spent the wartime years in his native Belarus, becoming one of the country's most distinguished artists and a member of the modernist avant-garde , founding the Vitebsk Arts College . He later worked in and near Moscow in difficult conditions during hard times in Russia following the Bolshevik Revolution , before leaving again for Paris in 1923. During World War II , he escaped occupied France to the United States, where he lived in New York City for seven years before returning to France in 1948.","{""h1"": ""Marc Chagall""}",[ 0.326 0.067 -0.061 -0.046 0.242 0.227 ... 0.153 -0.295 0.282 0.013 0.276 -0.331]
"Art critic Robert Hughes referred to Chagall as ""the quintessential Jewish artist of the twentieth century"". According to art historian Michael J. Lewis, Chagall was considered to be ""the last survivor of the first generation of European modernists"". For decades, he ""had also been respected as the world's pre-eminent Jewish artist"". [15] Using the medium of stained glass, he produced windows for the cathedrals of Reims and Metz as well as the Fraumünster in Zürich , windows for the UN and th ...... e experienced modernism's ""golden age"" in Paris, where ""he synthesized the art forms of Cubism , Symbolism , and Fauvism , and the influence of Fauvism ga